# SNS algorithm
Synthesizer of Normalized Signatures 

## Libraries 

In [5]:
import os
import warnings
import numpy as np
from math import comb
import synthesizer as sns

## Parameters 

In [2]:
# Map each alias-category from other datasets to WHITED
MATCHING_MAP = {"compactfluorescentlamp": "cfl", "vacuum": "vacuumcleaner",
                "waterkettle": "kettle", "hairiron": "flatiron",
                "coffeemaker": "coffeemachine",
                "incandescentlightbulb": "lightbulb", "airconditioner": "ac"}
# Datasets mixup and synthesization parameters
# Duration of each training/testing example (raw signal)
# in seconds:
SIGNAL_DURATION = 5
# Number of seconds from left and right sides of example's timeline, where
# appliance can't be switched off
FORBIDDEN_INTERVAL = 2.0
# Normalize all voltage magnitudes to the reference value
VOLTAGE_STANDARD = 311.0
# Normalize all frequencies to the same value (to avoid beats)
FREQUENCY_STANDARD = 50
# Thresholds
THD_THRESH = 0.1
ACTIVATION_THRESH = 0.1
# Rate after downsampling 
SAMPLING_RATE = 4000
# Random state for reproducability
RANDOM_STATE = 0

## Normalization and merging 

In [3]:
whited_dataset = sns.read_whited("/home/arx7ti/datasets/whited",
                                 downsampling=SAMPLING_RATE,
                                 voltage_standard=VOLTAGE_STANDARD,
                                 frequency_standard=FREQUENCY_STANDARD,
                                 thd_thresh=THD_THRESH,
                                 max_duration=SIGNAL_DURATION)
plaid_dataset = sns.read_plaid("/home/arx7ti/datasets/plaid",
                               downsampling=SAMPLING_RATE,
                               voltage_standard=VOLTAGE_STANDARD,
                               frequency_standard=FREQUENCY_STANDARD,
                               activation_thresh=ACTIVATION_THRESH,
                               thd_thresh=THD_THRESH,
                               max_duration=SIGNAL_DURATION,
                               matching_map=MATCHING_MAP)
patterns = sns.merge_datasets(whited_dataset, plaid_dataset)
sns.drop_empty_categories(patterns, inplace=True)

  0%|                                                                  | 2/1876 [00:00<01:43, 18.04it/s]

Dataset contains 1339 patterns from 55 categories
 ['fan', 'massage', 'lightbulb', 'mixer', 'flatiron', 'microwave', 'ledlight', 'iron', 'kettle', 'cablereceiver', 'benchgrinder', 'drillingmachine', 'laserprinter', 'washingmachine', 'halogen', 'gameconsole', 'ricecooker', 'hairdryer', 'mosquitorepellent', 'charger', 'airpump', 'shredder', 'toaster', 'coffeemachine', 'treadmill', 'powersupply', 'waterheater', 'tv', 'laptop', 'stove', 'networkswitch', 'vacuumcleaner', 'fridge', 'desktoppc', 'fanheater', 'monitor', 'multitool', 'hifi', 'solderingiron', 'halogenfluter', 'ac', 'deepfryer', 'shoewarmer', 'juicemaker', 'cfl', 'projector', 'sewingmachine', 'sandwichmaker', 'waterpump', 'desoldering', 'kitchenhood', 'heater', 'guitaramp', 'cablemodem', 'jigsaw']


100%|███████████████████████████████████████████████████████████████| 1876/1876 [03:42<00:00,  8.44it/s]
/home/arx7ti/tmp/cold-nilm/synthesizer.py:360: UserWarning: 208 patterns were skipped
  warnings.warn("%d patterns were skipped" % skips)


Dataset contains 1668 patterns from 16 categories
 ['cfl', 'fridge', 'hairdryer', 'ac', 'laptop', 'microwave', 'vacuumcleaner', 'lightbulb', 'fan', 'washingmachine', 'heater', 'coffeemachine', 'kettle', 'flatiron', 'solderingiron', 'blender']
15 categories intersects
 ['solderingiron', 'fridge', 'laptop', 'washingmachine', 'hairdryer', 'lightbulb', 'heater', 'fan', 'cfl', 'coffeemachine', 'kettle', 'flatiron', 'ac', 'microwave', 'vacuumcleaner']
56 categories in merged dataset
 ['fan', 'massage', 'lightbulb', 'mixer', 'flatiron', 'microwave', 'ledlight', 'iron', 'kettle', 'cablereceiver', 'benchgrinder', 'drillingmachine', 'laserprinter', 'washingmachine', 'halogen', 'gameconsole', 'ricecooker', 'hairdryer', 'mosquitorepellent', 'charger', 'airpump', 'shredder', 'toaster', 'coffeemachine', 'treadmill', 'powersupply', 'waterheater', 'tv', 'laptop', 'stove', 'networkswitch', 'vacuumcleaner', 'fridge', 'desktoppc', 'fanheater', 'monitor', 'multitool', 'hifi', 'solderingiron', 'halogenflut

## Save/load patterns 

In [13]:
try:
    patterns = np.load("./synthetic/patterns.npy", allow_pickle=True).item()
except:
    os.makedirs("synthetic", exist_ok=True)
    np.save("./synthetic/patterns.npy", patterns)

## Train/validation/test splits for each experiment 

### Subsets for Experiment 1

In [14]:
patterns_1 = sns.filter_categories(patterns, ["cfl", "fridge",  "ac", "laptop",
                                              "vacuumcleaner", "lightbulb",
                                              "fan", "coffeemachine", "kettle",
                                              "flatiron", "solderingiron"])
train_patterns_1, validation_patterns_1, test_patterns_1 = sns.split(
    patterns_1, ratios=[0.6, 0.1], random_state=RANDOM_STATE, verbose=False)

44 categories will be dropped.


### Subsets for Experiment 2

In [15]:
patterns_2 = sns.filter_categories(patterns, ["kettle", "microwave", 
                                          "washingmachine", "fridge",
                                          "vacuumcleaner"])
train_patterns_2, validation_patterns_2, test_patterns_2 = sns.split(
    patterns_2, ratios=[0.6, 0.1], random_state=RANDOM_STATE,verbose=False)

50 categories will be dropped.


### Subsets for Experiment 3

In [16]:
train_patterns_3, validation_patterns_3, test_patterns_3 = sns.split(
    patterns, ratios=[0.6, 0.1], random_state=RANDOM_STATE,verbose=False)

## Synthesization 

In [17]:
warnings.filterwarnings('ignore')

### Synthetic data for experiment 1

In [18]:
TRAIN_DATASET_PATH_1 = "./synthetic/train_1"
VALIDATION_DATASET_PATH_1 = "./synthetic/validation_1"
TEST_DATASET_PATH_1 = "./synthetic/test_1"

In [19]:
for i in range(1, 4):
    print("Combination bound for %d labels = %d" %
          (i, comb(len(train_patterns_1), i)))

Combination bound for 1 labels = 11
Combination bound for 2 labels = 55
Combination bound for 3 labels = 165


In [20]:
stats_1 = sns.get_stats(train_patterns_1, validation_patterns_1, test_patterns_1)
train_median_1, val_median_1, test_median_1 = stats_1["medians"]
train_size_1, val_size_1, test_size_1 = stats_1["sizes"]
stats_1

{'medians': (67, 11, 35), 'sizes': (864, 141, 440)}

In [21]:
# Limits for distribution and multiplicity of labels.
# Key is number of simultaneously working devices
# Value is tuple from combination limit and representation limit respectively
TRAIN_LIMITS_1 = {1: (train_size_1, train_size_1), 2: (
    55, 55*train_median_1), 3: (165, 165*train_median_1)}
VALIDATION_LIMITS_1 = {1: (val_size_1, val_size_1), 2: (
    55, 55*val_median_1), 3: (165, 165*val_median_1)}
TEST_LIMITS_1 = {1: (test_size_1, test_size_1), 2: (
    55, 55*test_median_1),  3: (165, 165*test_median_1)}

In [22]:
print("Train subset")
print("total reprs", sum([v[1] for v in TRAIN_LIMITS_1.values()]))
print("total combs", sum([v[0] for v in TRAIN_LIMITS_1.values()]))
print("reprs/comb", [v[1]/v[0] for v in TRAIN_LIMITS_1.values()])
print("\nValidation subset")
print("total reprs", sum([v[1] for v in VALIDATION_LIMITS_1.values()]))
print("total combs", sum([v[0] for v in VALIDATION_LIMITS_1.values()]))
print("reprs/comb", [v[1]/v[0] for v in VALIDATION_LIMITS_1.values()])
print("\nTest subset")
print("total reprs", sum([v[1] for v in TEST_LIMITS_1.values()]))
print("total combs", sum([v[0] for v in TEST_LIMITS_1.values()]))
print("reprs/comb", [v[1]/v[0] for v in TEST_LIMITS_1.values()])

Train subset
total reprs 15604
total combs 1084
reprs/comb [1.0, 67.0, 67.0]

Validation subset
total reprs 2561
total combs 361
reprs/comb [1.0, 11.0, 11.0]

Test subset
total reprs 8140
total combs 660
reprs/comb [1.0, 35.0, 35.0]


In [23]:
sns.build_dataset("train", train_patterns_1, TRAIN_LIMITS_1,
              collection_path=TRAIN_DATASET_PATH_1,
              signal_duration=SIGNAL_DURATION,
              forbidden_interval=FORBIDDEN_INTERVAL,
              voltage_magnitude=VOLTAGE_STANDARD,
              fundamental=FREQUENCY_STANDARD, sampling_rate=SAMPLING_RATE,
              p_fft=5, p_hop=1, verbose=True, to_specgrams=True,
              to_features=False, n_jobs=24, random_state=RANDOM_STATE)

Collection 1 is in progress...
[DISTRIBUTION] 11 combinations from 1 labels; 864 examples.
Collection 1 is ready.
Collection 2 is in progress...
[DISTRIBUTION] 55 combinations from 2 labels; 3685 examples.
[SKIP] # Collisions = 3 @ ('coffeemachine', 'fridge')
[SKIP] # Collisions = 11 @ ('flatiron', 'solderingiron')
[SKIP] # Collisions = 9 @ ('coffeemachine', 'solderingiron')
[SKIP] # Collisions = 10 @ ('coffeemachine', 'flatiron')
[SKIP] # Collisions = 1 @ ('coffeemachine', 'kettle')
[SKIP] # Collisions = 1 @ ('kettle', 'solderingiron')
[SKIP] # Collisions = 1 @ ('flatiron', 'fridge')
Collection 2 is ready.
Collection 3 is in progress...
[DISTRIBUTION] 165 combinations from 3 labels; 11055 examples.
[SKIP] # Collisions = 1 @ ('flatiron', 'fridge', 'solderingiron')
[SKIP] # Collisions = 1 @ ('coffeemachine', 'flatiron', 'solderingiron')
Collection 3 is ready.


In [24]:
sns.build_dataset("val", validation_patterns_1, VALIDATION_LIMITS_1,
              collection_path=VALIDATION_DATASET_PATH_1,
              signal_duration=SIGNAL_DURATION,
              forbidden_interval=FORBIDDEN_INTERVAL,
              voltage_magnitude=VOLTAGE_STANDARD,
              fundamental=FREQUENCY_STANDARD, sampling_rate=SAMPLING_RATE,
              p_fft=5, p_hop=1, verbose=True, to_specgrams=True,
              to_features=False, n_jobs=24, random_state=RANDOM_STATE)

Collection 1 is in progress...
[DISTRIBUTION] 11 combinations from 1 labels; 141 examples.
Collection 1 is ready.
Collection 2 is in progress...
[DISTRIBUTION] 55 combinations from 2 labels; 584 examples.
[SKIP] # Collisions = 1 @ ('kettle', 'solderingiron')
[SKIP] # Collisions = 1 @ ('fridge', 'solderingiron')
[SKIP] # Collisions = 1 @ ('flatiron', 'solderingiron')
[SKIP] # Collisions = 1 @ ('flatiron', 'kettle')
[SKIP] # Collisions = 1 @ ('coffeemachine', 'flatiron')
[SKIP] # Collisions = 1 @ ('coffeemachine', 'fridge')
Collection 2 is ready.
Collection 3 is in progress...
[DISTRIBUTION] 165 combinations from 3 labels; 1812 examples.
[SKIP] # Collisions = 1 @ ('coffeemachine', 'flatiron', 'kettle')
[SKIP] # Collisions = 1 @ ('coffeemachine', 'flatiron', 'fridge')
[SKIP] # Collisions = 3 @ ('coffeemachine', 'flatiron', 'solderingiron')
[SKIP] # Collisions = 1 @ ('coffeemachine', 'kettle', 'solderingiron')
[SKIP] # Collisions = 1 @ ('coffeemachine', 'fridge', 'kettle')
[SKIP] # Collisi

In [25]:
sns.build_dataset("test", test_patterns_1, TEST_LIMITS_1,
              collection_path=TEST_DATASET_PATH_1,
              signal_duration=SIGNAL_DURATION,
              forbidden_interval=FORBIDDEN_INTERVAL,
              voltage_magnitude=VOLTAGE_STANDARD,
              fundamental=FREQUENCY_STANDARD, sampling_rate=SAMPLING_RATE,
              p_fft=5, p_hop=1, verbose=True, to_specgrams=True,
              to_features=False, n_jobs=24, random_state=RANDOM_STATE)

Collection 1 is in progress...
[DISTRIBUTION] 11 combinations from 1 labels; 440 examples.
Collection 1 is ready.
Collection 2 is in progress...
[DISTRIBUTION] 55 combinations from 2 labels; 1925 examples.
[SKIP] # Collisions = 1 @ ('coffeemachine', 'kettle')
[SKIP] # Collisions = 11 @ ('coffeemachine', 'solderingiron')
[SKIP] # Collisions = 13 @ ('coffeemachine', 'flatiron')
[SKIP] # Collisions = 1 @ ('fridge', 'solderingiron')
[SKIP] # Collisions = 1 @ ('fridge', 'kettle')
[SKIP] # Collisions = 1 @ ('coffeemachine', 'fridge')
[SKIP] # Collisions = 9 @ ('flatiron', 'solderingiron')
[SKIP] # Collisions = 2 @ ('flatiron', 'fridge')
[SKIP] # Collisions = 1 @ ('flatiron', 'kettle')
[SKIP] # Collisions = 1 @ ('kettle', 'solderingiron')
Collection 2 is ready.
Collection 3 is in progress...
[DISTRIBUTION] 165 combinations from 3 labels; 5775 examples.
[SKIP] # Collisions = 1 @ ('coffeemachine', 'kettle', 'solderingiron')
Collection 3 is ready.


### Synthetic data for experiment 2

In [26]:
TRAIN_DATASET_PATH_2 = "./synthetic/train_2"
VALIDATION_DATASET_PATH_2 = "./synthetic/validation_2"
TEST_DATASET_PATH_2 = "./synthetic/test_2"

In [27]:
for i in range(1, 5):
    print("Combination bound for %d labels = %d" %
          (i, comb(len(train_patterns_2), i)))

Combination bound for 1 labels = 5
Combination bound for 2 labels = 10
Combination bound for 3 labels = 10
Combination bound for 4 labels = 5


In [28]:
stats_2 = sns.get_stats(train_patterns_2, validation_patterns_2, test_patterns_2)
train_median_2, val_median_2, test_median_2 = stats_2["medians"]
train_size_2, val_size_2, test_size_2 = stats_2["sizes"]
stats_2

{'medians': (57, 9, 29), 'sizes': (364, 58, 188)}

In [29]:
# Limits for distribution and multiplicity of labels.
# Key is number of simultaneously working devices
# Value is tuple from combination limit and representation limit respectively
TRAIN_LIMITS_2 = {1: (5, 5*train_median_2), 2: (10, 10*train_median_2),
                  3: (10, 10*train_median_2), 4: (5, 5*train_median_2)}
VALIDATION_LIMITS_2 = {1: (val_size_2, val_size_2), 2: (
    10, 10*val_median_2), 3: (10, 10*val_median_2), 4: (5, 5*val_median_2)}
TEST_LIMITS_2 = {1: (test_size_2, test_size_2), 2: (10, 10*test_median_2),
                 3: (10, 10*test_median_2), 4: (5, 5*test_median_2)}

In [30]:
print("Train subset")
print("total reprs", sum([v[1] for v in TRAIN_LIMITS_2.values()]))
print("total combs", sum([v[0] for v in TRAIN_LIMITS_2.values()]))
print("reprs/comb", [v[1]/v[0] for v in TRAIN_LIMITS_2.values()])
print("\nValidation subset")
print("total reprs", sum([v[1] for v in VALIDATION_LIMITS_2.values()]))
print("total combs", sum([v[0] for v in VALIDATION_LIMITS_2.values()]))
print("reprs/comb", [v[1]/v[0] for v in VALIDATION_LIMITS_2.values()])
print("\nTest subset")
print("total reprs", sum([v[1] for v in TEST_LIMITS_2.values()]))
print("total combs", sum([v[0] for v in TEST_LIMITS_2.values()]))
print("reprs/comb", [v[1]/v[0] for v in TEST_LIMITS_2.values()])

Train subset
total reprs 1710
total combs 30
reprs/comb [57.0, 57.0, 57.0, 57.0]

Validation subset
total reprs 283
total combs 83
reprs/comb [1.0, 9.0, 9.0, 9.0]

Test subset
total reprs 913
total combs 213
reprs/comb [1.0, 29.0, 29.0, 29.0]


In [31]:
sns.build_dataset("train", train_patterns_2, TRAIN_LIMITS_2,
              collection_path=TRAIN_DATASET_PATH_2,
              signal_duration=SIGNAL_DURATION,
              forbidden_interval=FORBIDDEN_INTERVAL,
              voltage_magnitude=VOLTAGE_STANDARD,
              fundamental=FREQUENCY_STANDARD, sampling_rate=SAMPLING_RATE,
              p_fft=5, p_hop=1, verbose=True, to_specgrams=True,
              to_features=False, n_jobs=24, random_state=RANDOM_STATE)

Collection 1 is in progress...
[DISTRIBUTION] 5 combinations from 1 labels; 284 examples.
Collection 1 is ready.
Collection 2 is in progress...
[DISTRIBUTION] 10 combinations from 2 labels; 570 examples.
Collection 2 is ready.
Collection 3 is in progress...
[DISTRIBUTION] 10 combinations from 3 labels; 570 examples.
Collection 3 is ready.
Collection 4 is in progress...
[DISTRIBUTION] 5 combinations from 4 labels; 285 examples.
Collection 4 is ready.


In [32]:
sns.build_dataset("val", validation_patterns_2, VALIDATION_LIMITS_2,
              collection_path=VALIDATION_DATASET_PATH_2,
              signal_duration=SIGNAL_DURATION,
              forbidden_interval=FORBIDDEN_INTERVAL,
              voltage_magnitude=VOLTAGE_STANDARD,
              fundamental=FREQUENCY_STANDARD, sampling_rate=SAMPLING_RATE,
              p_fft=5, p_hop=1, verbose=True, to_specgrams=True,
              to_features=False, n_jobs=24, random_state=RANDOM_STATE)

Collection 1 is in progress...
[DISTRIBUTION] 5 combinations from 1 labels; 58 examples.
Collection 1 is ready.
Collection 2 is in progress...
[DISTRIBUTION] 10 combinations from 2 labels; 90 examples.
Collection 2 is ready.
Collection 3 is in progress...
[DISTRIBUTION] 10 combinations from 3 labels; 90 examples.
Collection 3 is ready.
Collection 4 is in progress...
[DISTRIBUTION] 5 combinations from 4 labels; 45 examples.
Collection 4 is ready.


In [33]:
sns.build_dataset("test", test_patterns_2, TEST_LIMITS_2,
              collection_path=TEST_DATASET_PATH_2,
              signal_duration=SIGNAL_DURATION,
              forbidden_interval=FORBIDDEN_INTERVAL,
              voltage_magnitude=VOLTAGE_STANDARD,
              fundamental=FREQUENCY_STANDARD, sampling_rate=SAMPLING_RATE,
              p_fft=5, p_hop=1, verbose=True, to_specgrams=True,
              to_features=False, n_jobs=24, random_state=RANDOM_STATE)

Collection 1 is in progress...
[DISTRIBUTION] 5 combinations from 1 labels; 188 examples.
Collection 1 is ready.
Collection 2 is in progress...
[DISTRIBUTION] 10 combinations from 2 labels; 290 examples.
Collection 2 is ready.
Collection 3 is in progress...
[DISTRIBUTION] 10 combinations from 3 labels; 290 examples.
Collection 3 is ready.
Collection 4 is in progress...
[DISTRIBUTION] 5 combinations from 4 labels; 145 examples.
Collection 4 is ready.


### Synthetic data for experiment 3

In [34]:
TRAIN_DATASET_PATH_3 = "./synthetic/train_3"
VALIDATION_DATASET_PATH_3 = "./synthetic/validation_3"
TEST_DATASET_PATH_3 = "./synthetic/test_3"

In [35]:
for i in range(1, len(train_patterns_3)+1):
    print("Combination bound for %d labels = %d" %
          (i, comb(len(train_patterns_3), i)))

Combination bound for 1 labels = 55
Combination bound for 2 labels = 1485
Combination bound for 3 labels = 26235
Combination bound for 4 labels = 341055
Combination bound for 5 labels = 3478761
Combination bound for 6 labels = 28989675
Combination bound for 7 labels = 202927725
Combination bound for 8 labels = 1217566350
Combination bound for 9 labels = 6358402050
Combination bound for 10 labels = 29248649430
Combination bound for 11 labels = 119653565850
Combination bound for 12 labels = 438729741450
Combination bound for 13 labels = 1451182990950
Combination bound for 14 labels = 4353548972850
Combination bound for 15 labels = 11899700525790
Combination bound for 16 labels = 29749251314475
Combination bound for 17 labels = 68248282427325
Combination bound for 18 labels = 144079707346575
Combination bound for 19 labels = 280576272201225
Combination bound for 20 labels = 505037289962205
Combination bound for 21 labels = 841728816603675
Combination bound for 22 labels = 1300853625660225

In [36]:
stats_3 = sns.get_stats(train_patterns_3, validation_patterns_3, test_patterns_3)
train_median_3, val_median_3, test_median_3 = stats_3["medians"]
train_size_3, val_size_3, test_size_3 = stats_3["sizes"]
stats_3

{'medians': (12, 2, 6), 'sizes': (1800, 295, 912)}

In [37]:
# Limits for distribution and multiplicity of labels.
# Key is number of simultaneously working devices
# Value is tuple from combination limit and representation limit respectively
TRAIN_LIMITS_3 = {3: (2000, 2000*train_median_3),
                  4: (2000, 2000*train_median_3),
                  6: (4000, 4000*(train_median_3//2)),
                  8: (8000, 8000*(train_median_3//4)),
                  10: (8000, 8000*(train_median_3//4))}
VALIDATION_LIMITS_3 = {1: (val_size_3, val_size_3),
                       2: (2000, 2000*val_median_3),
                       3: (2000, 2000*val_median_3),
                       4: (2000, 2000*val_median_3),
                       5: (4000, 4000*(val_median_3//2)),
                       6: (4000, 4000*(val_median_3//2)),
                       7: (4000, 4000*(val_median_3//2)),
                       8: (4000, 4000*(val_median_3//2)),
                       9: (4000, 4000*(val_median_3//2)),
                       10: (4000, 4000*(val_median_3//2))}
TEST_LIMITS_3 = {1: (test_size_3, test_size_3),
                 2: (2000, 2000*test_median_3),
                 3: (2000, 2000*test_median_3),
                 4: (2000, 2000*test_median_3),
                 5: (4000, 4000*(test_median_3//2)),
                 6: (4000, 4000*(test_median_3//2)),
                 7: (4000, 4000*(test_median_3//2)),
                 8: (12000, 12000*(test_median_3//4)),
                 9: (12000, 12000*(test_median_3//4)),
                 10: (12000, 12000*(test_median_3//4))}

In [38]:
print("Train subset")
print("total reprs", sum([v[1] for v in TRAIN_LIMITS_3.values()]))
print("total combs", sum([v[0] for v in TRAIN_LIMITS_3.values()]))
print("reprs/comb", [v[1]/v[0] for v in TRAIN_LIMITS_3.values()])
print("\nValidation subset")
print("total reprs", sum([v[1] for v in VALIDATION_LIMITS_3.values()]))
print("total combs", sum([v[0] for v in VALIDATION_LIMITS_3.values()]))
print("reprs/comb", [v[1]/v[0] for v in VALIDATION_LIMITS_3.values()])
print("\nTest subset")
print("total reprs", sum([v[1] for v in TEST_LIMITS_3.values()]))
print("total combs", sum([v[0] for v in TEST_LIMITS_3.values()]))
print("reprs/comb", [v[1]/v[0] for v in TEST_LIMITS_3.values()])

Train subset
total reprs 120000
total combs 24000
reprs/comb [12.0, 12.0, 6.0, 3.0, 3.0]

Validation subset
total reprs 36295
total combs 30295
reprs/comb [1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

Test subset
total reprs 108912
total combs 54912
reprs/comb [1.0, 6.0, 6.0, 6.0, 3.0, 3.0, 3.0, 1.0, 1.0, 1.0]


In [49]:
sns.build_dataset("train", train_patterns_3, TRAIN_LIMITS_3,
                  collection_path=TRAIN_DATASET_PATH_3,
                  signal_duration=SIGNAL_DURATION,
                  forbidden_interval=FORBIDDEN_INTERVAL,
                  voltage_magnitude=VOLTAGE_STANDARD,
                  fundamental=FREQUENCY_STANDARD, sampling_rate=SAMPLING_RATE,
                  p_fft=5, p_hop=1, verbose=True, to_specgrams=True,
                  to_features=False, n_jobs=24, random_state=RANDOM_STATE)

Collection 1 is in progress...
[DISTRIBUTION] 2000 combinations from 3 labels; 24000 examples.
[SKIP] # Collisions = 1 @ ('juicemaker', 'networkswitch', 'projector')
[SKIP] # Collisions = 1 @ ('desktoppc', 'kitchenhood', 'projector')
[SKIP] # Collisions = 1 @ ('airpump', 'desoldering', 'projector')
[SKIP] # Collisions = 1 @ ('laserprinter', 'networkswitch', 'waterpump')
[SKIP] # Collisions = 1 @ ('deepfryer', 'fanheater', 'kitchenhood')
[SKIP] # Collisions = 2 @ ('desktoppc', 'juicemaker', 'projector')
[SKIP] # Collisions = 1 @ ('cablereceiver', 'desoldering', 'networkswitch')
[SKIP] # Collisions = 1 @ ('guitaramp', 'laserprinter', 'treadmill')
[SKIP] # Collisions = 2 @ ('fanheater', 'halogen', 'stove')
[SKIP] # Collisions = 1 @ ('cablemodem', 'desktoppc', 'hifi')
[SKIP] # Collisions = 1 @ ('cablereceiver', 'desoldering', 'laserprinter')
[SKIP] # Collisions = 1 @ ('desoldering', 'stove', 'treadmill')
[SKIP] # Collisions = 1 @ ('hifi', 'mosquitorepellent', 'networkswitch')
[SKIP] # Coll

In [40]:
sns.build_dataset("val", validation_patterns_3, VALIDATION_LIMITS_3,
              collection_path=VALIDATION_DATASET_PATH_3,
              signal_duration=SIGNAL_DURATION,
              forbidden_interval=FORBIDDEN_INTERVAL,
              voltage_magnitude=VOLTAGE_STANDARD,
              fundamental=FREQUENCY_STANDARD, sampling_rate=SAMPLING_RATE,
              p_fft=5, p_hop=1, verbose=True, to_specgrams=True,
              to_features=False, n_jobs=24, random_state=RANDOM_STATE)

Collection 1 is in progress...
[DISTRIBUTION] 55 combinations from 1 labels; 295 examples.
Collection 1 is ready.
Collection 2 is in progress...
[DISTRIBUTION] 1485 combinations from 2 labels; 3655 examples.
[SKIP] # Collisions = 1 @ ('drillingmachine', 'ricecooker')
[SKIP] # Collisions = 1 @ ('drillingmachine', 'solderingiron')
[SKIP] # Collisions = 1 @ ('flatiron', 'shredder')
[SKIP] # Collisions = 1 @ ('drillingmachine', 'shredder')
[SKIP] # Collisions = 1 @ ('monitor', 'shredder')
[SKIP] # Collisions = 1 @ ('shoewarmer', 'shredder')
[SKIP] # Collisions = 1 @ ('flatiron', 'monitor')
[SKIP] # Collisions = 1 @ ('monitor', 'solderingiron')
[SKIP] # Collisions = 1 @ ('ricecooker', 'shoewarmer')
[SKIP] # Collisions = 1 @ ('coffeemachine', 'shoewarmer')
Collection 2 is ready.
Collection 3 is in progress...
[DISTRIBUTION] 2000 combinations from 3 labels; 3835 examples.
Collection 3 is ready.
Collection 4 is in progress...
[DISTRIBUTION] 2000 combinations from 4 labels; 3931 examples.
Colle

In [41]:
sns.build_dataset("test", test_patterns_3, TEST_LIMITS_3,
              collection_path=TEST_DATASET_PATH_3,
              signal_duration=SIGNAL_DURATION,
              forbidden_interval=FORBIDDEN_INTERVAL,
              voltage_magnitude=VOLTAGE_STANDARD,
              fundamental=FREQUENCY_STANDARD, sampling_rate=SAMPLING_RATE,
              p_fft=5, p_hop=1, verbose=True, to_specgrams=True,
              to_features=False, n_jobs=24, random_state=RANDOM_STATE)

Collection 1 is in progress...
[DISTRIBUTION] 55 combinations from 1 labels; 912 examples.
Collection 1 is ready.
Collection 2 is in progress...
[DISTRIBUTION] 1485 combinations from 2 labels; 11880 examples.
[SKIP] # Collisions = 3 @ ('laserprinter', 'treadmill')
[SKIP] # Collisions = 1 @ ('desoldering', 'ricecooker')
[SKIP] # Collisions = 2 @ ('benchgrinder', 'jigsaw')
[SKIP] # Collisions = 3 @ ('desoldering', 'halogenfluter')
[SKIP] # Collisions = 2 @ ('multitool', 'networkswitch')
[SKIP] # Collisions = 2 @ ('mosquitorepellent', 'shredder')
[SKIP] # Collisions = 3 @ ('fanheater', 'laserprinter')
[SKIP] # Collisions = 1 @ ('halogen', 'networkswitch')
[SKIP] # Collisions = 1 @ ('guitaramp', 'networkswitch')
[SKIP] # Collisions = 3 @ ('sandwichmaker', 'waterpump')
[SKIP] # Collisions = 2 @ ('airpump', 'laserprinter')
[SKIP] # Collisions = 1 @ ('projector', 'solderingiron')
[SKIP] # Collisions = 2 @ ('desoldering', 'shredder')
[SKIP] # Collisions = 3 @ ('desoldering', 'networkswitch')
[

[SKIP] # Collisions = 2 @ ('cablemodem', 'deepfryer')
[SKIP] # Collisions = 2 @ ('deepfryer', 'halogenfluter')
[SKIP] # Collisions = 1 @ ('networkswitch', 'shoewarmer')
[SKIP] # Collisions = 2 @ ('benchgrinder', 'multitool')
[SKIP] # Collisions = 2 @ ('treadmill', 'waterpump')
[SKIP] # Collisions = 2 @ ('cablemodem', 'juicemaker')
[SKIP] # Collisions = 2 @ ('mosquitorepellent', 'waterpump')
[SKIP] # Collisions = 1 @ ('shoewarmer', 'shredder')
[SKIP] # Collisions = 1 @ ('cablemodem', 'drillingmachine')
[SKIP] # Collisions = 2 @ ('airpump', 'cablemodem')
[SKIP] # Collisions = 3 @ ('desoldering', 'waterpump')
[SKIP] # Collisions = 4 @ ('networkswitch', 'stove')
[SKIP] # Collisions = 1 @ ('multitool', 'waterpump')
[SKIP] # Collisions = 2 @ ('monitor', 'mosquitorepellent')
[SKIP] # Collisions = 2 @ ('jigsaw', 'networkswitch')
[SKIP] # Collisions = 2 @ ('cablereceiver', 'sewingmachine')
[SKIP] # Collisions = 1 @ ('desktoppc', 'solderingiron')
[SKIP] # Collisions = 2 @ ('halogen', 'sandwichma

[SKIP] # Collisions = 2 @ ('deepfryer', 'fanheater')
[SKIP] # Collisions = 3 @ ('fanheater', 'halogen')
[SKIP] # Collisions = 3 @ ('fanheater', 'hifi')
[SKIP] # Collisions = 4 @ ('benchgrinder', 'sewingmachine')
[SKIP] # Collisions = 3 @ ('desktoppc', 'sewingmachine')
[SKIP] # Collisions = 2 @ ('fanheater', 'kitchenhood')
[SKIP] # Collisions = 1 @ ('sewingmachine', 'treadmill')
[SKIP] # Collisions = 1 @ ('airpump', 'solderingiron')
[SKIP] # Collisions = 2 @ ('juicemaker', 'kitchenhood')
[SKIP] # Collisions = 4 @ ('desoldering', 'laserprinter')
[SKIP] # Collisions = 2 @ ('kitchenhood', 'projector')
[SKIP] # Collisions = 1 @ ('deepfryer', 'shoewarmer')
[SKIP] # Collisions = 1 @ ('guitaramp', 'ricecooker')
[SKIP] # Collisions = 2 @ ('projector', 'sewingmachine')
[SKIP] # Collisions = 3 @ ('projector', 'stove')
[SKIP] # Collisions = 2 @ ('halogen', 'kitchenhood')
[SKIP] # Collisions = 3 @ ('benchgrinder', 'juicemaker')
[SKIP] # Collisions = 1 @ ('sandwichmaker', 'treadmill')
[SKIP] # Colli

[SKIP] # Collisions = 1 @ ('fanheater', 'networkswitch', 'waterpump')
[SKIP] # Collisions = 1 @ ('benchgrinder', 'desoldering', 'mosquitorepellent')
[SKIP] # Collisions = 1 @ ('halogen', 'treadmill', 'waterpump')
[SKIP] # Collisions = 1 @ ('deepfryer', 'kitchenhood', 'laserprinter')
[SKIP] # Collisions = 1 @ ('desktoppc', 'sandwichmaker', 'stove')
[SKIP] # Collisions = 1 @ ('cablemodem', 'desoldering', 'hifi')
[SKIP] # Collisions = 1 @ ('desoldering', 'juicemaker', 'multitool')
[SKIP] # Collisions = 1 @ ('airpump', 'desktoppc', 'sewingmachine')
[SKIP] # Collisions = 1 @ ('jigsaw', 'juicemaker', 'stove')
[SKIP] # Collisions = 1 @ ('desoldering', 'networkswitch', 'waterpump')
[SKIP] # Collisions = 2 @ ('laserprinter', 'multitool', 'networkswitch')
[SKIP] # Collisions = 1 @ ('mosquitorepellent', 'sandwichmaker', 'sewingmachine')
[SKIP] # Collisions = 1 @ ('halogenfluter', 'networkswitch', 'sandwichmaker')
Collection 3 is ready.
Collection 4 is in progress...
[DISTRIBUTION] 2000 combinatio